# Lab 8 - Integer Programming - BnB for MIP

Information on group members:

1) 156034, Wojciech Bogacz <br>
2) 156039, Kzrysztof Skrobała

In [1]:
from pulp import *
import numpy as np
import pandas as pd

1) Given is the below MIP problem. Note that the first 5 variables are of an integer type with specified upper bounds

In [2]:
def getProblem(relaxed = False):
    
    A = [
        [0,3,2,0,0,0,-3,-1,0,0],
        [1,1,0,2,0,0,0,-1,2,1],
        [0,0,2,-2,3,0,-2,2,1,0],
        [0,0,2,0,0,-1,0,0,0,1],
        [0,2,0,0,0,-2,0,0,0,1],
        [1,4,0,0,0,0,-3,6,2,0],
        [2,2,0,0,2,2,0,0,2,2],
        [0,0,3,0,-1,1,0,-1,0,1],
        [0,0,0,0,5,0,1,1,0,3],
        [2,-7,0,0,0,1,0,8,2,0]]
    b = [10,15,20,20,30,50,40,20,25,25]
    c = [5, 7, 5, 5, 5, 5, 7, 4, 9, 10]
    uB = [5, 8, 4, 5, 4, 5, 5, 3, 3, 3]
    
    problem = LpProblem(name="bnb-problem", sense=LpMaximize)
    
    ### 5 integers and 3 continuous (if relaxed, 8 cont.)
    cat = ['Integer' for i in range(5)] + ['Continuous' for i in range(5)]
    if relaxed: cat = ['Continuous' for i in range(5)] + ['Continuous' for i in range(5)]
        
    x = [LpVariable(name="x"+ str(i+1), lowBound=0, upBound=uB[i], cat = cat[i]) for i in range(10)]
    
    for r in range(10):
        expr = lpSum([x[j] * A[r][j] for j in range(10)])
        problem += LpConstraint(e=expr, sense = -1, name = "baseC"+str(r+1), rhs = b[r])
        
    obj_func = lpSum([x[j] * c[j] for j in range(10)])
    problem += obj_func
    
    return x, problem

x, P = getProblem()
print(P)

bnb-problem:
MAXIMIZE
5*x1 + 10*x10 + 7*x2 + 5*x3 + 5*x4 + 5*x5 + 5*x6 + 7*x7 + 4*x8 + 9*x9 + 0
SUBJECT TO
baseC1: 3 x2 + 2 x3 - 3 x7 - x8 <= 10

baseC2: x1 + x10 + x2 + 2 x4 - x8 + 2 x9 <= 15

baseC3: 2 x3 - 2 x4 + 3 x5 - 2 x7 + 2 x8 + x9 <= 20

baseC4: x10 + 2 x3 - x6 <= 20

baseC5: x10 + 2 x2 - 2 x6 <= 30

baseC6: x1 + 4 x2 - 3 x7 + 6 x8 + 2 x9 <= 50

baseC7: 2 x1 + 2 x10 + 2 x2 + 2 x5 + 2 x6 + 2 x9 <= 40

baseC8: x10 + 3 x3 - x5 + x6 - x8 <= 20

baseC9: 3 x10 + 5 x5 + x7 + x8 <= 25

baseC10: 2 x1 - 7 x2 + x6 + 8 x8 + 2 x9 <= 25

VARIABLES
0 <= x1 <= 5 Integer
x10 <= 3 Continuous
0 <= x2 <= 8 Integer
0 <= x3 <= 4 Integer
0 <= x4 <= 5 Integer
0 <= x5 <= 4 Integer
x6 <= 5 Continuous
x7 <= 5 Continuous
x8 <= 3 Continuous
x9 <= 3 Continuous



2) The below function returns None if the problem has no feasible solutions. Otherwise, it returns a tuple: objective function values and a vector of decision variables. 

In [3]:
def getSolution(x, problem):
    status = problem.solve()
    if problem.status != 1: return None
    return problem.objective.value(), [_.value() for _ in x]

3) PuLP can solve MIP problems. Hence, the "relaxed" flag can be set to False. Solve the problem and analyze the obtained outcome.  

In [4]:
x, problem = getProblem(relaxed = False)
print(getSolution(x, problem))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wojtek/Documents/uczelnia/oper_research/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/af337e095b814268abcabe6199ebaa3b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/af337e095b814268abcabe6199ebaa3b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 83 RHS
At line 94 BOUNDS
At line 105 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 211.333 - 0.00 seconds
Cgl0004I processed model has 7 rows, 10 columns (5 integer (0 of which binary)) and 36 elements
Cbc0012I Integer solution of -206 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0012I Integer solution of -207 found by DiveCoefficient after 158 iterations and 0 nodes (0.05 seconds)
Cbc003

4) Now, compare this solution with the one obtained for the relaxed LP problem: 

In [5]:
x, problem = getProblem(relaxed = True)
print(getSolution(x, problem))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wojtek/Documents/uczelnia/oper_research/.venv/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/406f71617ba84d5fa3554bbe46840766-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/406f71617ba84d5fa3554bbe46840766-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 73 RHS
At line 84 BOUNDS
At line 95 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-3) rows, 10 (0) columns and 36 (-11) elements
0  Obj -0 Dual inf 64.49999 (10)
6  Obj 211.33333
Optimal - objective value 211.33333
After Postsolve, objective 211.33333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 211.3333333 - 6 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU s

5) Your task is to implement the Branch and Bound Algorithm for solving MIP problems. You can use the PuLP library for solving the relaxed LP subproblems. <br>
<ul> 
    <li> Firstly, as a node selection policy, implement the default DFS-like strategy as shown in the lecture (generate both children in one iteration; prioritize the left children, i.e., associated with the "<=" constraint). As for the variable selection policy, take the one with the lowest index  (default, arbitrary selection). 
<li> Identify how many LP relaxed problems have to be solved to find the optimum. Note that such a number was reported to be 35 for the default policies. However, it may vary slightly for different solvers due to possible multiple sub-optima.
<li> Propose at least 3 new node and variable selection policies with the aim of minimizing the number of solver runs required to reach the optimum.  Try getting below 20. 
    </ul>

In [14]:
def getSolution(problem):
    status = problem.solve(PULP_CBC_CMD(msg=False))
    if problem.status != 1: return None
    x = problem.variables()
    return problem.objective.value(), [_.value() for _ in x]

In [8]:
def check_int(problem, x, x_labels):
    x_var = problem.variables()
    for i in range(len(x_var)):
        if x_labels[str(x_var[i])] == "Integer":
            if x[i] % 1 != 0:
                return False
    return True

In [9]:
def get_node(stos, BFS = False):
    if BFS:
        return stos.pop(0)
    return stos.pop()

In [10]:
def sort_var(x_weights, x_var):
    x_var = sorted(x_var, key=lambda x: -1 * x_weights[str(x)])
    return x_var

In [75]:
stos = []
import copy
def branch_and_bound(problem, x_labels, stos, BFS = False, order = False):

    best = 0
    best_x = None

    stos.append(copy.deepcopy(problem))

    licznik = 2
    while len(stos) > 0:
        licznik += 1

        p = get_node(stos, BFS)
        res = getSolution(p)

        if res is None: # if problem is infeasible, skip
            continue

        solution, x = res
        if solution <= best: # if solution is worse than best, skip
            continue

        if check_int(p, x, x_labels): # if solution is integer and better than best, update best
            if solution > best:
                best = solution
                best_x = x
            continue

        x_var = p.variables()

        if order:
            for i in range(len(x_var)-1, -1, -1):
               if x_labels[str(x_var[i])] == "Integer":
                    if x_var[i].value() % 1 != 0:

                        pr = copy.deepcopy(p)
                        pr.variables()[i].lowBound = x_var[i].value()//1 + 1
                        stos.append(pr)

                        pr = copy.deepcopy(p)
                        pr.variables()[i].upBound = x_var[i].value()//1
                        stos.append(pr)
                        break
        else:
            for i in range(len(x_var)):
                if x_labels[str(x_var[i])] == "Integer":
                    if x_var[i].value() % 1 != 0:

                        pr = copy.deepcopy(p)
                        pr.variables()[i].lowBound = x_var[i].value()//1 + 1
                        stos.append(pr)
    
                        pr = copy.deepcopy(p)
                        pr.variables()[i].upBound = x_var[i].value()//1
                        stos.append(pr)
                        break

    return best, best_x, licznik

In [76]:
x_labels = {"x1": "Integer", "x2": "Integer", "x3": "Integer", "x4": "Integer", "x5": "Integer", "x6": "Continuous", "x7": "Continuous", "x8": "Continuous", "x9": "Continuous", "x10": "Continuous"}
x, problem = getProblem(relaxed = True)

In [77]:
print(branch_and_bound(problem, x_labels,  stos, BFS=False, order=False))

(207.0, [3.0, 3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0], 35)


In [78]:
print(branch_and_bound(problem, x_labels,  stos, BFS=True, order=False))

(207.0, [3.0, 3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0], 43)


In [79]:
print(branch_and_bound(problem, x_labels,  stos, BFS=True, order=True))

(207.0, [3.0, 3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0], 25)


In [80]:
print(branch_and_bound(problem, x_labels,  stos, BFS=False, order=True))


(207.0, [3.0, 3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0], 19)
